In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2022.parquet
/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2021.parquet
/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2018.csv
/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2018.parquet
/kaggle/input/flight-delay-dataset-20182022/Airlines.csv
/kaggle/input/flight-delay-dataset-20182022/readme.md
/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2022.csv
/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2020.csv
/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2020.parquet
/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2019.csv
/kaggle/input/flight-delay-dataset-20182022/readme.html
/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2021.csv
/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2019.parquet
/kaggle/input/flight-delay-dataset-20182022/raw/Flights_2018_7.csv
/kaggle/input/flight-delay-dataset-20182022/ra

# Introduction

### What we will be doing with this dataset:
- We will be using this dataset to effectively and accurately predict length of time for flight delays. We will use various regression and machine learning techniques to learn from the training data and then test our prediction models with our test data.

### Why we chose this dataset:
- We chose the given dataset as it is both rich and robust. The size allows us to adjust various factors in order to observe common phenomena. The number of columns allows us to explore which variables affect flight delays; whereas, the number of rows esnure we have enough data to make good judgements.

### What results we hope to achieve:
- We aim to predict the length of flight delays with high confidence.

In [2]:
df_flights_2018 = pd.read_csv("/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2018.csv")
df_flights_2018.dropna(inplace = True)
#df_flights_2019 = pd.read_csv("/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2019.csv")
#df_flights_2019.dropna(inplace = True)

#let's look at our code after running this
#let's look at the code
print(df_flights_2018.head())
print(df_flights_2018.info())

   FlightDate            Airline Origin Dest  Cancelled  Diverted  CRSDepTime  \
0  2018-01-23  Endeavor Air Inc.    ABY  ATL      False     False        1202   
1  2018-01-24  Endeavor Air Inc.    ABY  ATL      False     False        1202   
2  2018-01-25  Endeavor Air Inc.    ABY  ATL      False     False        1202   
3  2018-01-26  Endeavor Air Inc.    ABY  ATL      False     False        1202   
4  2018-01-27  Endeavor Air Inc.    ABY  ATL      False     False        1400   

   DepTime  DepDelayMinutes  DepDelay  ...  WheelsOff  WheelsOn  TaxiIn  \
0   1157.0              0.0      -5.0  ...     1211.0    1249.0     7.0   
1   1157.0              0.0      -5.0  ...     1210.0    1246.0    12.0   
2   1153.0              0.0      -9.0  ...     1211.0    1251.0    11.0   
3   1150.0              0.0     -12.0  ...     1207.0    1242.0    11.0   
4   1355.0              0.0      -5.0  ...     1412.0    1448.0    11.0   

   CRSArrTime  ArrDelay  ArrDel15  ArrivalDelayGroups  ArrTime

In [3]:
#only working with 2018 flights for now to work faster on modeling and code
#df_flights = pd.concat([df_flights_2018, df_flights_2019])

In [4]:
#let's look at the code
print(df_flights_2018.head())
print(df_flights_2018.info())

   FlightDate            Airline Origin Dest  Cancelled  Diverted  CRSDepTime  \
0  2018-01-23  Endeavor Air Inc.    ABY  ATL      False     False        1202   
1  2018-01-24  Endeavor Air Inc.    ABY  ATL      False     False        1202   
2  2018-01-25  Endeavor Air Inc.    ABY  ATL      False     False        1202   
3  2018-01-26  Endeavor Air Inc.    ABY  ATL      False     False        1202   
4  2018-01-27  Endeavor Air Inc.    ABY  ATL      False     False        1400   

   DepTime  DepDelayMinutes  DepDelay  ...  WheelsOff  WheelsOn  TaxiIn  \
0   1157.0              0.0      -5.0  ...     1211.0    1249.0     7.0   
1   1157.0              0.0      -5.0  ...     1210.0    1246.0    12.0   
2   1153.0              0.0      -9.0  ...     1211.0    1251.0    11.0   
3   1150.0              0.0     -12.0  ...     1207.0    1242.0    11.0   
4   1355.0              0.0      -5.0  ...     1412.0    1448.0    11.0   

   CRSArrTime  ArrDelay  ArrDel15  ArrivalDelayGroups  ArrTime

In [5]:
#Section 2: Initial Data Cleaning and Wrangling

#1. Change FlightDate to datetime time
df_flights_2018["FlightDate"] = pd.to_datetime(df_flights_2018["FlightDate"])

#2. Change Airline type to String
df_flights_2018["Airline"] = df_flights_2018["Airline"].astype(str)

#3. Change Origin and Dest type to String
df_flights_2018["Origin"] = df_flights_2018["Origin"].astype(str)
df_flights_2018["Dest"] = df_flights_2018["Dest"].astype(str)

#Now we want to remove any 'known' values that could predict the outcome. For example, DepDel15 would indicate
# we're going to be delayed and bias the model. So we're going to drop all these values that we would
# not know in advance while we're waiting for a plane to take off. This includes all the arrival values, 
# diverted values, cancelled values, etc... We also remove a lot of the redudant location information to make the dataset
# more manageable
keeper_columns = ["Quarter","DayOfWeek", "FlightDate",
                 "Airline",
                 "Origin", "OriginState",
                 "Dest", "DestState",
                  "CRSDepTime","Distance", "DepDelay"]

#Let's now copy the df to be a clean one to work with
df_flights_2018_clean = df_flights_2018[keeper_columns].copy()

#DepDelay is going to be our initial target for prediction. Let's rename that as dep_min_delay
df_flights_2018_clean.rename(columns={"DepDelay": "dep_min_delay"}, inplace=True)

#CRSDepTime and DepTimeBlk are tought to understand. Let's rename those
df_flights_2018_clean.rename(columns={"CRSDepTime": "schedule_dep_time", "DepTimeBlk": "scheduled_dep_block"}, inplace=True)

#we can see that we have a bit more cleaning to do for changing types
#let's convert the type of our target value - dep_min_delay - to int and round it
df_flights_2018_clean["dep_min_delay"] = df_flights_2018_clean["dep_min_delay"].round().astype(int)

#let's convert distance to int as well
df_flights_2018_clean["Distance"] = df_flights_2018_clean["Distance"].astype(int)

#Adjusting our date time fields (year, month, day)
# Handling datetime data (if applicable)
df_flights_2018_clean['Year'] = df_flights_2018_clean['FlightDate'].dt.year
df_flights_2018_clean['Month'] = df_flights_2018_clean['FlightDate'].dt.month
df_flights_2018_clean['Day'] = df_flights_2018_clean['FlightDate'].dt.day

In [6]:
#to save memory we now delete our original dataset
#del df_flights_2018
#gc.collect()

In [7]:
print(df_flights_2018_clean.head())
print(df_flights_2018_clean.info())

   Quarter  DayOfWeek FlightDate            Airline Origin OriginState Dest  \
0        1          2 2018-01-23  Endeavor Air Inc.    ABY          GA  ATL   
1        1          3 2018-01-24  Endeavor Air Inc.    ABY          GA  ATL   
2        1          4 2018-01-25  Endeavor Air Inc.    ABY          GA  ATL   
3        1          5 2018-01-26  Endeavor Air Inc.    ABY          GA  ATL   
4        1          6 2018-01-27  Endeavor Air Inc.    ABY          GA  ATL   

  DestState  schedule_dep_time  Distance  dep_min_delay  Year  Month  Day  
0        GA               1202       145             -5  2018      1   23  
1        GA               1202       145             -5  2018      1   24  
2        GA               1202       145             -9  2018      1   25  
3        GA               1202       145            -12  2018      1   26  
4        GA               1400       145             -5  2018      1   27  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5578618 entries, 0 

In [8]:
#we need to reduce the dataset further before modeling. It currently includes all airports in the USA which is just too much to one hot encode. So we're going to reduce it down.

#1. Let's see which airports are the most common 
origin_counts = df_flights_2018_clean['Origin'].value_counts()
top_origins = origin_counts.head(20)
print(top_origins)

ORD    282770
DEN    249288
ATL    221242
LAX    181472
IAH    161187
SFO    154833
EWR    145580
LAS    143054
SEA    130912
DFW    129181
PHX    117148
MCO    115943
BOS    112322
CLT    109738
DTW     98097
LGA     95210
BWI     95197
MSP     88652
FLL     86451
IAD     84340
Name: Origin, dtype: int64


In [9]:
# now let's get our top 20 destinations
dest_counts = df_flights_2018_clean['Dest'].value_counts()
top_dest = dest_counts.head(20)
print(top_dest)

ORD    282130
DEN    248875
ATL    221544
LAX    181687
IAH    160930
SFO    154863
EWR    145303
LAS    143180
SEA    130771
DFW    128629
PHX    116948
MCO    115777
BOS    112434
CLT    109519
DTW     98349
BWI     95016
LGA     95002
MSP     88739
FLL     86389
IAD     84278
Name: Dest, dtype: int64


In [10]:
#Now that we have our list of top origins and destinations, we are going to reduce it further by only including these airports
#how many flights do we currently have in the dataset?
print(df_flights_2018_clean.shape[0])
#our original clean dataset has 5.5 million rows
#first we filter for origins
df_flights_2018_clean_reduced = df_flights_2018_clean[df_flights_2018_clean['Origin'].isin(top_origins.index)]
print(df_flights_2018_clean_reduced.shape[0])
#this gets us down to 2.8 million rows
#now we filter down further based on top 20 destination
df_flights_2018_clean_reduced = df_flights_2018_clean_reduced[df_flights_2018_clean_reduced['Dest'].isin(top_dest.index)]
print(df_flights_2018_clean_reduced.shape[0])
#great! Now we are down to 1.1 million rows. We've reduced our dataset by 80 percent

5578618
2802617
1113556


In [11]:
#Let's check out what our new dataset looks like
#how many different airlines are represented?
print("Number of unique values for Airline:", df_flights_2018_clean_reduced['Airline'].nunique())
#Ok, we still have 22 different airlines represented. This is good.

Number of unique values for Airline: 22


In [12]:
#What does our geographic dispersion look like? Let's see how many different states are represented
print("Number of unique states in OriginState:", df_flights_2018_clean_reduced['OriginState'].nunique())
#Good we have 17 states represented, that's some diversity.
print("Number of unique states in DestState:", df_flights_2018_clean_reduced['OriginState'].nunique())
#17 as well for the Destinations. This should work decently. 

Number of unique states in OriginState: 17
Number of unique states in DestState: 17


In [13]:
#Let's visualize our states to see where the bulk of the data is coming from
origin_state_counts = df_flights_2018_clean_reduced['OriginState'].value_counts()
print(origin_state_counts.head(10))
dest_state_counts = df_flights_2018_clean_reduced['DestState'].value_counts()
print(dest_state_counts.head(10))

#We see here then that our rankings are the same, the top 10 origin states and top 10 departure states are the same. It makes sense since airlines use
#hub and spoke models and these hubs would be the busiest airports

CA    158250
TX    105217
FL     95069
IL     85900
CO     80246
GA     74158
NJ     63546
MA     60570
NV     59809
WA     49749
Name: OriginState, dtype: int64
CA    158704
TX    104995
FL     95108
IL     85881
CO     79683
GA     74245
NJ     63498
MA     60685
NV     59910
WA     49758
Name: DestState, dtype: int64


In [14]:
#Now let's see which airports have the most departure delays

#group by Origin and we want the dep_min_delay to get an average for all flights coming out of that airport
airport_delays = df_flights_2018_clean_reduced.groupby('Origin')['dep_min_delay'].mean()
#now we sort by descending value
airport_delays_sorted = airport_delays.sort_values(ascending=False)
print(airport_delays_sorted.head(20))

#Interesting. So EWR (Newark) has the worst average delay with 14.7 minutes, then Dallas is second with 13.3 mins of delay. 
#Seattle is the best of our major airports with only 8.03 mins of average delay.

Origin
EWR    14.674928
DFW    13.319274
BOS    13.168962
MCO    13.036935
ORD    13.016205
LGA    12.379286
FLL    11.677273
DEN    11.602846
BWI    11.398214
ATL    10.758785
SFO    10.727685
CLT    10.578674
IAD    10.262088
PHX    10.126002
IAH    10.058599
LAS     9.966460
DTW     8.506716
MSP     8.290735
LAX     8.150424
SEA     8.036724
Name: dep_min_delay, dtype: float64


In [15]:
#let's dig into the Newark data more to find out why the delays are so bad.
#which airlines at Newark have the worst delays?
#get the flights only from Newark
flights_from_EWR = df_flights_2018_clean_reduced[df_flights_2018_clean_reduced['Origin'] == 'EWR']

#groupby airline and rank the average delay
EWR_delays = flights_from_EWR.groupby('Airline')['dep_min_delay'].mean()
print(EWR_delays.sort_values(ascending=False))

#Wow! Hopefully you're not flying Commutiar or JetBlue anytime soon out of Newark!

Airline
Commutair Aka Champlain Enterprises, Inc.    31.350067
JetBlue Airways                              25.567400
GoJet Airlines, LLC d/b/a United Express     19.441989
SkyWest Airlines Inc.                        18.765672
United Air Lines Inc.                        14.901459
Southwest Airlines Co.                       13.744661
Spirit Air Lines                             13.642779
Endeavor Air Inc.                            13.151982
ExpressJet Airlines Inc.                     12.306306
Delta Air Lines Inc.                          9.832812
American Airlines Inc.                        8.583433
Alaska Airlines Inc.                          8.130397
Republic Airlines                             7.815070
Virgin America                                6.675728
Envoy Air                                     5.500000
Name: dep_min_delay, dtype: float64


In [16]:
#Section 4: Prep for modeling
#First lets remind ourselves what our current dataset looks like
print(df_flights_2018_clean_reduced.head())
print(df_flights_2018_clean_reduced.info())

     Quarter  DayOfWeek FlightDate            Airline Origin OriginState Dest  \
305        1          1 2018-01-01  Endeavor Air Inc.    ATL          GA  IAH   
365        1          2 2018-01-02  Endeavor Air Inc.    DTW          MI  DFW   
399        1          2 2018-01-02  Endeavor Air Inc.    DTW          MI  LGA   
452        1          3 2018-01-10  Endeavor Air Inc.    DTW          MI  IAD   
453        1          3 2018-01-10  Endeavor Air Inc.    IAD          VA  DTW   

    DestState  schedule_dep_time  Distance  dep_min_delay  Year  Month  Day  
305        TX               1151       689            141  2018      1    1  
365        TX               1540       986              7  2018      1    2  
399        NY               1733       502             14  2018      1    2  
452        VA                720       383             -8  2018      1   10  
453        MI                944       383              2  2018      1   10  
<class 'pandas.core.frame.DataFrame'>
Int64In

In [17]:
#Section 4: Prep for Modeling
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder

#One hot encoding before modeling
#In order to effectively model, we need to one-hot-encode five values:
#"Airline" (22 values)
#"Origin" (20 values)
#"OriginState" (20 values)
#"Dest"(17 values)
#"DestState" (17 values)

#4.1 encode "Airline"
#create an encoder instance
encoder = OneHotEncoder(drop='first', sparse_output=False)
#reshape the column 
airline_data = df_flights_2018_clean_reduced[['Airline']]
airline_data_reshaped = airline_data.values.reshape(-1, 1)
#encode with fit_transform
encoded_airline = pd.DataFrame(encoder.fit_transform(airline_data_reshaped), columns=encoder.get_feature_names_out(['Airline']))
#concatenate bback with originl
df_flights_2018_encoded = pd.concat([df_flights_2018_clean_reduced, encoded_airline], axis=1)
#drop the original airline column
df_flights_2018_encoded.drop(columns=['Airline'], inplace=True)

#we also need to drop the datetime column
df_flights_2018_encoded.drop(columns=['FlightDate'], inplace=True)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [18]:
print(df_flights_2018_encoded.head())
print(df_flights_2018_encoded.info())

     Quarter  DayOfWeek Origin OriginState Dest DestState  schedule_dep_time  \
305      1.0        1.0    ATL          GA  IAH        TX             1151.0   
365      1.0        2.0    DTW          MI  DFW        TX             1540.0   
399      1.0        2.0    DTW          MI  LGA        NY             1733.0   
452      1.0        3.0    DTW          MI  IAD        VA              720.0   
453      1.0        3.0    IAD          VA  DTW        MI              944.0   

     Distance  dep_min_delay    Year  ...  Airline_Horizon Air  \
305     689.0          141.0  2018.0  ...                  0.0   
365     986.0            7.0  2018.0  ...                  0.0   
399     502.0           14.0  2018.0  ...                  0.0   
452     383.0           -8.0  2018.0  ...                  0.0   
453     383.0            2.0  2018.0  ...                  0.0   

     Airline_JetBlue Airways  Airline_Mesa Airlines Inc.  \
305                      1.0                         0.0   
36

In [19]:
#Great! One of our five categorical columns is encoded. Now lets do the last five.
#4.2 encode "Origin"
encoder = OneHotEncoder(drop='first', sparse_output=False)
origin_data = df_flights_2018_clean_reduced[['Origin']]
origin_data_reshaped = origin_data.values.reshape(-1, 1)
encoded_airline = pd.DataFrame(encoder.fit_transform(origin_data_reshaped), columns=encoder.get_feature_names_out(['Origin']))
df_flights_2018_encoded = pd.concat([df_flights_2018_encoded, encoded_airline], axis=1)
df_flights_2018_encoded.drop(columns=['Origin'], inplace=True)

#4.3 encode "OriginState"
encoder = OneHotEncoder(drop='first', sparse_output=False)
originState_data = df_flights_2018_clean_reduced[['OriginState']]
originState_data_reshaped = originState_data.values.reshape(-1, 1)
encoded_airline = pd.DataFrame(encoder.fit_transform(originState_data_reshaped), columns=encoder.get_feature_names_out(['OriginState']))
df_flights_2018_encoded = pd.concat([df_flights_2018_encoded, encoded_airline], axis=1)
df_flights_2018_encoded.drop(columns=['OriginState'], inplace=True)

#4.4 encode "Dest"
encoder = OneHotEncoder(drop='first', sparse_output=False)
dest_data = df_flights_2018_clean_reduced[['Dest']]
dest_data_reshaped = dest_data.values.reshape(-1, 1)
encoded_airline = pd.DataFrame(encoder.fit_transform(dest_data_reshaped), columns=encoder.get_feature_names_out(['Dest']))
df_flights_2018_encoded = pd.concat([df_flights_2018_encoded, encoded_airline], axis=1)
df_flights_2018_encoded.drop(columns=['Dest'], inplace=True)

#4.5 encode "DestState"
encoder = OneHotEncoder(drop='first', sparse_output=False)
destState_data = df_flights_2018_clean_reduced[['DestState']]
destState_data_reshaped = destState_data.values.reshape(-1, 1)
encoded_airline = pd.DataFrame(encoder.fit_transform(destState_data_reshaped), columns=encoder.get_feature_names_out(['DestState']))
df_flights_2018_encoded = pd.concat([df_flights_2018_encoded, encoded_airline], axis=1)
df_flights_2018_encoded.drop(columns=['DestState'], inplace=True)


print(df_flights_2018_encoded.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2007634 entries, 305 to 1113555
Data columns (total 99 columns):
 #   Column                                             Dtype  
---  ------                                             -----  
 0   Quarter                                            float64
 1   DayOfWeek                                          float64
 2   schedule_dep_time                                  float64
 3   Distance                                           float64
 4   dep_min_delay                                      float64
 5   Year                                               float64
 6   Month                                              float64
 7   Day                                                float64
 8   Airline_Alaska Airlines Inc.                       float64
 9   Airline_American Airlines Inc.                     float64
 10  Airline_Comair Inc.                                float64
 11  Airline_Commutair Aka Champlain Enterprises, Inc

In [20]:
#we need to fill all the nan values as 0 to do linear regression. Otherwise we get an error because linear regression can't handle Nan
df_flights_2018_encoded = df_flights_2018_encoded.fillna(0)

In [21]:
#The last thing we need to do is prep our datasets! We need train and test values for x's and y's
#Our x value (input features) will be everything EXCEPT dep_min_delay
X = df_flights_2018_encoded.drop(columns=['dep_min_delay'])

#Our y values (prediction targets) will be the dep_min_delay value
y = df_flights_2018_encoded['dep_min_delay']

#We split training and testing with an 80% / 20% split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [22]:
#Section 5: Linear Regression
# We will initially do four linear regression comparisons across our data to see how each performs
#1. Simple Linear Regression
#2. L1 Lasso Regression
#3. L2 Ridge Regression
#4. Elastic Net Regression
# We will get the MSE and the R^2 for each to compare

#no need to do any code in this cell, it's just a placeholder

In [23]:
df_flights_2018_encoded = df_flights_2018_encoded.dropna()
print(df_flights_2018_encoded.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2007634 entries, 305 to 1113555
Data columns (total 99 columns):
 #   Column                                             Dtype  
---  ------                                             -----  
 0   Quarter                                            float64
 1   DayOfWeek                                          float64
 2   schedule_dep_time                                  float64
 3   Distance                                           float64
 4   dep_min_delay                                      float64
 5   Year                                               float64
 6   Month                                              float64
 7   Day                                                float64
 8   Airline_Alaska Airlines Inc.                       float64
 9   Airline_American Airlines Inc.                     float64
 10  Airline_Comair Inc.                                float64
 11  Airline_Commutair Aka Champlain Enterprises, Inc

In [24]:
#Section 5.1: Simple Linear Regression
#import model
from sklearn.linear_model import LinearRegression

#create the model
lr_model = LinearRegression()

#fit the model
lr_model.fit(X_train, y_train)

#make predictions on the X_test inputs
y_pred = lr_model.predict(X_test)

#get the mse using our model prediction values (y_pred) vs the actual y values (y_test)
mse = mean_squared_error(y_test, y_pred)

#get the r^2 the same way
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R-squared:", r2)

MSE: 1060.573956012466
R-squared: 0.04367785328314888


In [25]:
#MSE: 1060.573956012466
#R-squared: 0.04367785328314888
#Ok, so this model performs pretty poorly. While it's MSE is hard to interpret, an R^2 of 0.04 is pretty bad. Let's see if L1 regression is any better

In [26]:
#Section 5.2 L1 Lasso Regression
#import model and tools
from sklearn.linear_model import Lasso

#create the alpha value and lasso_reg model
#set our initial alpha_value to 0.1
alpha_value = 0.1

#make the model
l1_model = Lasso(alpha=alpha_value)

#fit the model
l1_model.fit(X_train, y_train)

#make predictions
y_pred = l1_model.predict(X_test)

#get the mse and r^2
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("MSE:", mse)
print("R-squared:", r2)

MSE: 1062.1813544183494
R-squared: 0.042228458183986683


In [27]:
#oh wow this model isn't much better. 0.04 again and it looks like it might have gotten worse
#maybe ridge regression will help?

In [28]:
#Section 5.3 L2 Ridge Regression
#import model
from sklearn.linear_model import Ridge

#create rr_model and set the alpha
alpha_value = 1.0
l2_model = Ridge(alpha=alpha_value)

# fit the model
l2_model.fit(X_train, y_train)

#make predictions
y_pred = l2_model.predict(X_test)

#get the mse and r^2
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("MSE:", mse)
print("R-squared:", r2)

MSE: 1060.573839378257
R-squared: 0.04367795845250244


In [29]:
#Yup not much better. We're going to really need to work on this dataset I think to make a decent model.

In [30]:
#Section 5.4 Elastic Net Regression
#import model
from sklearn.linear_model import ElasticNet

#create en_model with alpha, L1_ratio

#fit the model

#make predictions

#get the mse and r^2

In [31]:
#5.5 Summarize our findings
#Do the MSEs change across all four models?
#Do the R^2 change across all four models?
#Is one model clearly performing better? Why or why not?

In [32]:
#Section 6: Advanced Linear Regression
#Now we're going to do the same four models but at an advanced level with more tuning and tweaking
from sklearn.preprocessing import StandardScaler

#Section 6.0 Feature scaling to the dataset. We want to standardize/normalize the input features with probably StandardScaler (vs MinMaxScaler)
#build a scaler. Then we're going to also do 5-fold cross validation for each model. This combination: scaling + cross validation : should result
# in better performance across our models


#Section 6.1: Linear Regression w/ scaling and Cross-Validation
#fit_transform our dataset for modeling
#should be something like this:
#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X)
#Get new datasets
#X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=23)
#make a new model
#linear_reg = LinearRegression()
#do CV with 5 folds
#cv_vals = cross_val_score(linear_reg, X_scaled, y, cv=5, scoring='neg_mean_squared_error')

#get the average MSE across all 5 folds

#get the average R^2 across all 5 folds

In [33]:
#Section 6.2: L1 Lasso Regression with scaling and cross validation

In [34]:
#Section 6.3: L2 Ridge Regression with scaling and cross validation

In [35]:
#Section 6.4 Elastic Net Regression with scaling and cross validation

In [36]:
#Section 7 (if we have time)
#if time allows in the project for Section 7 we can do SKLearn Gradient Boosting Regression (GBR) and see how it compares

In [37]:
#Section 8 (if we have time)
#we can do PyTorch NN regression modeling to see how it compares